## Project 1: Weather Data and Flights Data

#### GROUP: Jing Sun, Holly Huo

#### Flights data
In this project, we were given two data sets: flights and weather2. The first data set: flights.csv, provides us with information about airline flights in and out of all of the New York airports between January 1, 2013 and December 31, 2013. In this data set, each row is an observation about a single flight, but there are a number of derived columns that need to be cleaned up in order to transform this data set into tidy form. For the transformation process, we only kept the month, day, carrier, flight number, departure hour (originally called hour), departure minute (originally called minute), origin and destination airports, departure delay time, arrival delay time, distance and air time. The other columns such as the actual departure time and actual arrival time could all be derived from the information kept in the table. For easy exporting purposes, we set the index column to 'month'.

In [1]:
import pandas as pd
flights = pd.read_csv("flights.csv", index_col = 'month') # import flights data set, and set month to be the index col
weather = pd.read_csv("weather2.csv")                     # import weather data set

In [2]:
flights_tidy = flights.loc[:,['day','carrier','flight','hour','minute','origin','dest','dep_delay','arr_delay','distance','air_time']]     
# filter out necessary columns
flights_tidy.rename(columns={flights_tidy.columns[3]: "dep_hour", flights_tidy.columns[4]: "dep_minute"},inplace=True) 
# rename the column hour and minute     
flights_tidy.head()
flights_tidy.to_csv('flights_tidy.csv')

#### Weather data
The second data set: weather2.csv, is data obtained from Weather Underground for weather metrics in Boston, MA over the time period ranging from January 1, 2013 and December 31, 2013, same time period as in the flights data. This data does not conform to our Tidy data constraints.

By examining the original weather2 data set, we can see that for every month, the data frame is composed of columns including year, month, measure, and each day of the month. If we look closer at the matrix composed of the 19 measures and corresponding data for each day of the month, we can see that in order to make it conform to our tidy data constraints, we can simply transform the matrix and transform the events column into four boolean columns by each weather event.

To transform the matrices month by month, we constructed a for loop to first select the matrix we need using iloc (indices of columns are scaled according to the number of days in each month, and indices of rows are selected for each 19 rows interval). After we transform the matrix, we now have days as a column. However, all of the days entries have an X attached to the front, so we applied a lambda function to remove the "X" to each entry in the days column. We then insert a "month" column using the value "iterator + 1" to each matrix, and concatenate the current matrix to the last transformed matrix. The codes for the above steps are shown below. (Note: the transformation of the events column is described in the next markdown cell)

In [3]:
month_num_of_days = [31,28,31,30,31,30,31,31,30,31,30,31]    # construct a list of number of days in each month
for i in range(12):                                          # iterate through all 12 months
    weather_current = weather.iloc[19*i:19*(i+1), 3: month_num_of_days[i]+4]   # select the matrix for current month
    weather_current.index = weather_current.iloc[:,0]        # set measure to be the index column
    del weather_current['measure']                           # delete the original measure column
    weather_current = weather_current.transpose()            # transform the matrix so we have days as the index col
    weather_current.index.name = 'day'                       # change the name of index col to day
    weather_current.reset_index(inplace = True)              # reset index to pull "day" out as a single column
    weather_current.day = weather_current.day.apply(lambda n : n[1:])  # remove "X" in front of each entry in day
    weather_current.insert(0, 'month', i+1)                  # insert a month column using the value "i+1"
    weather_current = weather_current.set_index('month')     # set the created month column to be the new index col
    if i == 0:
        weather_almost_tidy = weather_current                # set final data frame to be the current one for Jan
    else:                                                    # for months Feb to Dec
        weather_almost_tidy = pd.concat([weather_almost_tidy,weather_current])
        # concatenate current data frame to the final one
weather_almost_tidy.head()

measure,day,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,Max.Dew.PointF,MeanDew.PointF,Min.DewpointF,Max.Humidity,Mean.Humidity,Min.Humidity,Max.Sea.Level.PressureIn,Mean.Sea.Level.PressureIn,Min.Sea.Level.PressureIn,Max.VisibilityMiles,Mean.VisibilityMiles,Min.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Wind.SpeedMPH,PrecipitationIn,Events
month,,,,,,,,,,,,,,,,,,,,
1,1,37,29,20,24,16,4,64,53,41,29.93,29.79,29.73,10,10,10,28,14,0,NaN
1,2,30,21,12,12,6,-2,58,53,47,30.1,29.98,29.9,10,10,10,24,14,T,NaN
1,3,25,16,7,10,-1,-7,62,46,29,30.14,30.08,30,10,10,10,22,11,0,NaN
1,4,38,30,21,22,18,11,69,59,48,29.97,29.89,29.81,10,10,10,26,13,0,NaN
1,5,42,35,27,23,18,15,69,52,34,30.26,30.1,29.86,10,10,10,21,9,0,NaN


Now we have arrived to a much better data frame in terms of tidiness. The only problem is that the "Events" column contains many variables (Rain, Snow, Fog and Thunderstorm), which makes the data frame still not tidy. In order to separate each variable out to have its own column, we performed the following operations. We first select out the "Events" column as a new data frame, and iterate through every row of this new data frame to split it by "-" into a list of variables. The key idea here is to evaluate the list of variables for each day (row) and create a list of boolean values in the order of "Rain, Fog, Snow, Thunderstorm", and append each row list to a larger list to form a list of lists for us to construct a new data frame, with index being days and column labels being each weather event. After we obtain the new data frame (df_events) with four boolean event columns, we then concatenate the almost tidy data frame with this new one, and delete the original "Events" column to arrive to our final data frame (weather_tidy). This final data frame now conforms to our tidy data constraints.

In [4]:
weather_events = weather_almost_tidy[['Events']]        # take 'Events' column to construct a new data frame
df_lol = []                                             # create an empty list of lists
label = [1]*31 + [2]*28 + [3]*31 + [4]*30 + [5]*31 + [6]*30 + [7]*31 + [8]*31 + [9]*30 + [10]*31 + [11]*30 + [12]*31 # create a list of days for the entire year
col = ['Rain','Fog','Snow','Thunderstorm']              # create column labels
for lab,row in weather_events.iterrows():               # iterate through every row of the new data frame
    events = str(row[-1]).split('-')                    # split each row by "-" to give a list of event variables
    rowlist = []                                        # create an empty row list
    rowlist.append('Rain' in events)                    # append boolean values to row list
    rowlist.append('Fog' in events)
    rowlist.append('Snow' in events)
    rowlist.append('Thunderstorm' in events)
    df_lol.append(rowlist)                              # append row list to the list of lists
df_events = pd.DataFrame(df_lol, columns = col, index = label)      # create a new dataframe using lol

weather_tidy = pd.concat([weather_almost_tidy,df_events],axis=1)    # concatenate the almost tidy df with the new df
del weather_tidy['Events']                              # delete the original non-tidy 'Events' column
weather_tidy.head()
weather_tidy.to_csv('weather_tidy.csv')

#### Weather + flights data for BOS airport
In order to investigate the correlation between weather and flights delay time, we extracted the information including day, departure delay and arrival delay for the flights that have Boston as their destination using a loc function. Since there are many observations of delay times per day, we performed a groupby operation to calculate the mean of the delay times for each day, so that we have one observation of departure delay and arrival delay per day.

In [32]:
import numpy as np
flights_bos_by_day = flights_tidy.loc[flights_tidy.dest == 'BOS',['day','dep_delay','arr_delay']] # select useful rows and cols
flights_bos_by_day.reset_index(inplace=True)           # reset index to pull the column "month" out
flights_bos_by_day = flights_bos_by_day.groupby(by = ["month","day"]).agg(np.mean) # groupby month and day, take mean of delay times
flights_bos_by_day.reset_index(inplace=True)           # reset index to pull month and day out 
flights_bos_by_day.index = flights_bos_by_day["month"] # set month to be the index for exporting purpose
del flights_bos_by_day["month"]                        # delete original month column
flights_bos_by_day.head()

,day,dep_delay,arr_delay
month,,,
1,1,8.800000,-3.800000
1,2,4.884615,-5.653846
1,3,3.200000,-10.300000
1,4,8.566667,-0.800000
1,5,7.523810,-1.523810


In [31]:
writer = pd.ExcelWriter('BOS.xlsx', engine='xlsxwriter')
weather_tidy.to_excel(writer, sheet_name='Weather')
flights_bos_by_day.to_excel(writer, sheet_name='Flights_BOS')
writer.save()      # write two sheets weather_tidy and flights_bos_by_day to an excel file

### Relationship between destinations and average arrival delays

In the picture below, the size of the circle at each destination represents the number of airlines arrived, and the color of each circle represents the average time of arrival delay in minute. The range of the arrival delay time is from 22 minutes earlier to 38.16 minutes later than the scheduled arrive time. 
   
As the picture shows, the long-distance flights to the west coast tends to arrive early, and the flights to the east coast are more likely to arrive late. 
- Due to the long distance, speed is more adjustable for pilots and they can control time better. 
- The weather in west coast and Southern America is not very changable, and the east coast has more rain and snow throughout the year, these climatic factors may influence the arrive time. 

We can also find,
- There are more circles in east coast and mid-America. Since this data is about airline flights in and out of all of the New York airports, and New York is located in east coast, airlines can fly to more airports near New York. Due to the distance, airlines departured from New York mostly arrive at some airports located in large cities like LAX and SFO. So the size of circles in west coast are larger.

<img src="flights.jpg" width="900">

### Relationship between departure delays and arrival delays
The yellow line represents the average departure delay time in each month throughout the year, and the blue line represents the average arrival delay time in each month throughout the year. 

The trends of these two lines are very similar which means as long as the flight departures late, this flight is very likely to arrive late. But overall the average arrival delay time is shorter than the average departure delay time. It is obvious that in June, July and December, the average delay time is longer than the others. This may be influenced by the weather in these months. In boston, it rains the most in June and July and snows the most in December.

<img src="average_delay_time_by_month.jpg" width="800">

### Relationship between origins and average departure delays
The size of the circle represents the number of airlines departured from the airport, and the color of each circle represents the average departure delay time at each airport. The range of the departure delay time is from 10 minutes to 14.7 minutes later than the scheduled departure time. 

The second pictures shows number of flights of each airline at these three airports. 
According to the picture,
- The sizes of circles are about the same. This means the amount of airlines departured from each airport is about the same. 
- The color of each circle shows the average departure delay time of flights departured from EWR is the longest and LGA is the shortest. As we can see in the second picture, the majority of flights departured from EWR are from United airlines and Express Jet airlines. And JetBlue airlines takes up most flights departured from JFK. However the amount of flights from each airline is more evenly distributed in LGA. This guarantees the delay time of LGA will not be influenced by a certain airline.

<img src="avg_dep_delay_by_airports.jpg" width="850">

<img src="num_flights_by_airport_carrier.jpg" width="750">

### Relationship between time of the year and weather event
The picture below shows the weather event in each month throughout the year. The red line represents thunder, the orange line represents snow, the green line represents fog and the blue line represents rain. 

This data obtained from Weather Underground for weather metrics in Boston. Boston is under humid continental climate:
- Summer in boston is often hot and humid and winter in boston is cold(sometimes severely cold). 
- Precipitation is relatively well distributed year-round in many areas with this climate, while others may see a marked reduction in wintry precipitation. Snowfall is more common than rain during the height of winter. As we can see from May to July, about half of the month is rainy. After August, the number of rainy days decreases. After october, the number of the snowy days increases. 
- Most summer rainfall occurs during thunderstorms and a very occasional tropical system. So in summer, the most humid time in boston, the number of days with thunderstrom reaches the peak. 
- Evaporation fog is caused by cold air passing over warmer water or moist land. Snow is formed when temperatures are low and there is moisture in the atmosphere. So when it snows in the winter, it is also likely to have fog. Fog normally occurs at a relative humidity near 100%. So in summer, the most humid time of the year, fog is very likely to form.

<img src="weather_events_by_month.jpg" width="800">

### Relationship between weather events and average arrival and departure time
The graph below shows the relationship between weather events and average arrival and departure delay time by month for Boston airport (BOS) in the year of 2013. We can see that the trend of average departure delay time follows closely with the trend of weather events. In the months of June and July, there is a large number of raining days, which aligns closely with the increase in the average departure delay time, and the significant increase in the average arrival delay time. In the months of September to November, there are less weather events occurring, which again aligns closely with the significant decrease in both the average departure and arrival delay time. These observations allow us to conclude that weather events do play an important role in affecting flights delay time for BOS airport in the year of 2013.

<img src="flights_weather_BOS.jpg">
